In [1]:
import os
import tensorflow as tf
import cv2
import numpy as np
import time
import inspect
from keras.utils import to_categorical

/home/him/miniconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [3]:
files = ['jpg/' + x  for x in os.listdir('jpg/')]
labels = np.loadtxt('labels.csv', dtype='int32', delimiter=',')

In [4]:
files

['jpg/image_03293.jpg',
 'jpg/image_04651.jpg',
 'jpg/image_07387.jpg',
 'jpg/image_06031.jpg',
 'jpg/image_05377.jpg',
 'jpg/image_04075.jpg',
 'jpg/image_02861.jpg',
 'jpg/image_08059.jpg',
 'jpg/image_02845.jpg',
 'jpg/image_02617.jpg',
 'jpg/image_07593.jpg',
 'jpg/image_07716.jpg',
 'jpg/image_04370.jpg',
 'jpg/image_01670.jpg',
 'jpg/image_03476.jpg',
 'jpg/image_06285.jpg',
 'jpg/image_06121.jpg',
 'jpg/image_06752.jpg',
 'jpg/image_06966.jpg',
 'jpg/image_01610.jpg',
 'jpg/image_04353.jpg',
 'jpg/image_01114.jpg',
 'jpg/image_02386.jpg',
 'jpg/image_01298.jpg',
 'jpg/image_02268.jpg',
 'jpg/image_05732.jpg',
 'jpg/image_02321.jpg',
 'jpg/image_01518.jpg',
 'jpg/image_00826.jpg',
 'jpg/image_05501.jpg',
 'jpg/image_01243.jpg',
 'jpg/image_04824.jpg',
 'jpg/image_02646.jpg',
 'jpg/image_02072.jpg',
 'jpg/image_00532.jpg',
 'jpg/image_07907.jpg',
 'jpg/image_03248.jpg',
 'jpg/image_07030.jpg',
 'jpg/image_03059.jpg',
 'jpg/image_08180.jpg',
 'jpg/image_00554.jpg',
 'jpg/image_0045

In [5]:
filelabel = [ labels[int((x.split('_')[-1]).split('.')[0]) - 1] for x in files]

In [6]:
filelabel

[8,
 48,
 94,
 93,
 22,
 12,
 56,
 91,
 56,
 84,
 95,
 97,
 36,
 82,
 30,
 61,
 64,
 1,
 34,
 82,
 36,
 46,
 43,
 74,
 41,
 42,
 43,
 44,
 81,
 87,
 74,
 85,
 58,
 75,
 88,
 100,
 65,
 39,
 92,
 62,
 88,
 88,
 32,
 60,
 56,
 58,
 27,
 80,
 99,
 82,
 16,
 89,
 4,
 8,
 74,
 6,
 62,
 95,
 88,
 51,
 12,
 89,
 56,
 54,
 89,
 40,
 38,
 48,
 43,
 83,
 88,
 3,
 51,
 74,
 44,
 45,
 15,
 81,
 56,
 81,
 46,
 93,
 90,
 98,
 7,
 35,
 74,
 44,
 78,
 51,
 95,
 31,
 50,
 89,
 77,
 91,
 37,
 81,
 89,
 51,
 94,
 79,
 14,
 91,
 50,
 62,
 15,
 94,
 62,
 53,
 47,
 26,
 77,
 73,
 4,
 30,
 15,
 68,
 92,
 59,
 29,
 99,
 12,
 7,
 56,
 100,
 66,
 82,
 95,
 95,
 51,
 63,
 45,
 49,
 37,
 71,
 73,
 89,
 95,
 77,
 82,
 89,
 46,
 39,
 65,
 71,
 46,
 100,
 32,
 102,
 58,
 88,
 80,
 74,
 36,
 84,
 46,
 41,
 102,
 83,
 80,
 52,
 1,
 37,
 29,
 58,
 82,
 75,
 90,
 38,
 84,
 58,
 94,
 11,
 2,
 51,
 84,
 44,
 41,
 16,
 94,
 57,
 8,
 100,
 55,
 66,
 77,
 8,
 83,
 69,
 82,
 2,
 80,
 56,
 97,
 76,
 91,
 54,
 75,
 49,
 81,
 95

In [7]:
data = []
for file in files:
    data.append(cv2.resize(cv2.imread(file, 1), (224, 224)))

cat_labels = to_categorical(filelabel)
data = np.array(data)

In [8]:
len(data)

8189

In [9]:
data.shape

(8189, 224, 224, 3)

In [10]:
cv2.imshow('image', data[0])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
print(len(set(labels)), cat_labels.shape)

102 (8189, 103)


In [12]:
learning_rate = 0.001
num_steps = 200
batch_size = 128
display_step = 10

In [13]:
num_classes = 103
dropout = 0.5

In [46]:
X = tf.placeholder(tf.float32, [None, 224, 224, 3])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32)

In [47]:
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [48]:
def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

In [57]:
def conv_net(x, weights, biases, dropout):
    
    with tf.variable_scope('condsfdsffdsfv3', reuse=False) as scope:
        conv = tf.layers.conv2d(
            inputs=x,
            filters=32,
            kernel_size=[3, 3],
            padding='SAME',
            activation=tf.nn.relu
        )
        pool = tf.layers.max_pooling2d(conv, pool_size=[2, 2], strides=2, padding='SAME')
    
    with tf.variable_scope('cofdfdsfdsffnv4', reuse=False) as scope:
        conv = tf.layers.conv2d(
            inputs=pool,
            filters=128,
            kernel_size=[3, 3],
            padding='SAME',
            activation=tf.nn.relu
        )
        pool = tf.layers.max_pooling2d(conv, pool_size=[2, 2], strides=2, padding='SAME')
        
    print(pool.get_shape().as_list())    
    with tf.variable_scope('fully_cfdsffdsfsdsfonnected1', reuse=False) as scope:
        flat = tf.reshape(pool, [-1, 56*56*128])
        fc = tf.layers.dense(inputs=flat, units=512, activation=tf.nn.relu)
        drop = tf.layers.dropout(fc, rate=dropout)
        softmax = tf.layers.dense(inputs=drop, units=num_classes, activation=tf.nn.softmax, name=scope.name)
        
    return softmax

In [58]:
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 3, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, num_classes]))
}

In [59]:
biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [60]:
logits = conv_net(X, weights, biases, keep_prob)

[None, 56, 56, 128]


In [61]:
#tf.reset_default_graph()

In [62]:
prediction = tf.nn.softmax(logits)

In [63]:
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))

In [64]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

In [65]:
train_op = optimizer.minimize(loss_op)

In [66]:
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [67]:
init = tf.global_variables_initializer()

In [68]:
def batch_generator(X, y, batch_size, shuffle):
    #chenglong code for fiting from generator (https://www.kaggle.com/c/talkingdata-mobile-user-demographics/forums/t/22567/neural-network-for-sparse-matrices)
    number_of_batches = np.ceil(X.shape[0]/batch_size)
    counter = 0
    sample_index = np.arange(X.shape[0])
    if shuffle:
        np.random.shuffle(sample_index)
    while True:
        batch_index = sample_index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X[batch_index,:]
        y_batch = y[batch_index]
        counter += 1
        yield X_batch, y_batch
        if (counter == number_of_batches):
            if shuffle:
                np.random.shuffle(sample_index)
            counter = 0

In [69]:
datagen = batch_generator(data, cat_labels, 128, True)

In [70]:
with tf.Session() as sess:
    
    sess.run(init)
    
    for step in range(1, num_steps + 1):
        batch_x, batch_y = next(datagen)
        print(batch_x.shape, batch_y.shape)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: 0.8})
        if step % display_step == 0 or step == 1:
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x, Y: batch_y, keep_prob: 1.0})
            print("Step " + str(step) + ", Minibatch Loss= " + "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))
    print("Optimization Finished")

(128, 224, 224, 3) (128, 103)
Step 1, Minibatch Loss= 4.6278, Training Accuracy= 0.023
(128, 224, 224, 3) (128, 103)
(128, 224, 224, 3) (128, 103)
(128, 224, 224, 3) (128, 103)
(128, 224, 224, 3) (128, 103)
(128, 224, 224, 3) (128, 103)
(128, 224, 224, 3) (128, 103)
(128, 224, 224, 3) (128, 103)
(128, 224, 224, 3) (128, 103)
(128, 224, 224, 3) (128, 103)
Step 10, Minibatch Loss= 4.6513, Training Accuracy= 0.000
(128, 224, 224, 3) (128, 103)
(128, 224, 224, 3) (128, 103)
(128, 224, 224, 3) (128, 103)
(128, 224, 224, 3) (128, 103)
(128, 224, 224, 3) (128, 103)
(128, 224, 224, 3) (128, 103)
(128, 224, 224, 3) (128, 103)
(128, 224, 224, 3) (128, 103)
(128, 224, 224, 3) (128, 103)
(128, 224, 224, 3) (128, 103)
Step 20, Minibatch Loss= 4.6435, Training Accuracy= 0.008
(128, 224, 224, 3) (128, 103)
(128, 224, 224, 3) (128, 103)
(128, 224, 224, 3) (128, 103)
(128, 224, 224, 3) (128, 103)
(128, 224, 224, 3) (128, 103)
(128, 224, 224, 3) (128, 103)
(128, 224, 224, 3) (128, 103)
(128, 224, 224, 3

KeyboardInterrupt: 

In [30]:
temp = tf.InteractiveSession()

In [31]:
batch_x, batch_y = next(datagen)

In [33]:
batch_x = tf.cast(batch_x, tf.float32)

In [34]:
logits = conv_net(batch_x, weights, biases, keep_prob)

In [35]:
logits.get_shape().as_list()

[8192, 103]